In [2]:
import argparse
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [ ]:
CHROMA_PATH = "chroma"

PROMPT_TEMPLATE = """

"""

## Load Database

emb